In [213]:
%load_ext autoreload
%autoreload 2
import os, sys
os.environ['STREAM_LOG_LEVEL'] = 'ERROR'
os.environ['FILE_LOG_LEVEL'] = 'DEBUG'
os.environ['PROPAGATE_TO_ROOT_LOGGER'] = 'False'
os.environ['PROPAGATE_TO_ROOT_LOGGER'], os.environ['STREAM_LOG_LEVEL']
from trade.assets.Stock import Stock
from trade.assets.Option import Option
from trade.assets.OptionStructure import OptionStructure
from trade.helpers.Context import Context, clear_context
from trade.helpers.helper import (change_to_last_busday, 
                                  is_USholiday, 
                                  is_busday, 
                                  setup_logger, 
                                  generate_option_tick, 
                                  get_option_specifics_from_key)
from scipy.stats import percentileofscore
from dbase.DataAPI.ThetaData import list_contracts, retrieve_openInterest, retrieve_eod_ohlc
from pandas.tseries.offsets import BDay
import pandas as pd
from copy import deepcopy
import numpy as np
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
sys.path
from dbase.DataAPI.ThetaData import list_contracts
list_contracts('AAPL', '2025-01-13')

,root,expiration,strike,right
0,AAPL,20250919,215.0,P
1,AAPL,20250919,215.0,C
2,AAPL,20251017,215.0,P
3,AAPL,20250321,350.0,C
4,AAPL,20250124,220.0,P
...,...,...,...,...
1278,AAPL,20250131,217.5,C
1279,AAPL,20250718,215.0,C
1280,AAPL,20250718,215.0,P
1281,AAPL,20250815,215.0,P


In [78]:
class RiskManager:
    def __init__(self,
                 bars,
                 events,
                 initial_capital,
                 ):
        self.bars = bars
        self.events = events
        self.initial_capital = initial_capital
        self.symbol_list = self.bars.symbol_list


    def get_order(self, symbol, date, order_settings):
        pass


In [79]:
def generate_option_order(
        direction,
        dte,
        relative_strike,
):
    pass



In [80]:
generate_option_tick?

Signature: generate_option_tick(symbol, right, exp, strike)
Docstring: <no docstring>
File:      ~/cloned_repos/QuantTools/trade/helpers/helper.py
Type:      function

In [81]:
## Why is AMZN during cheap vol returning two?
def chain_details(date, ticker, tgt_dte, tgt_moneyness, right = 'P', moneyness_width = 0.15, print_stderr = False):
    return_dataframe = pd.DataFrame()
    errors = {}
    if not (is_USholiday(date) and not is_busday(date)):
        try:
            print(date, ticker) if print_stderr else None
            ## Get both calls and puts per moneyness. For 1 Moneyness, both will most be available. If not, if one is False, other True. 
            ## We will need to get two rows. 
            with Context(end_date = date):
                start_time = time.time()
                Stock_obj = Stock(ticker)
                end_time = time.time()
                print(f"Time taken to get stock object: {end_time-start_time}") if print_stderr else None
                Option_Chain = Stock_obj.option_chain()
                Option_Chain_Filtered = Option_Chain[Option_Chain[right.upper()] == True]
                Spot = Stock_obj.spot(ts = False)
                Spot = list(Spot.values())[0]
                if right == 'P':
                    Option_Chain_Filtered['relative_moneyness']  = Option_Chain_Filtered.index.get_level_values('strike')/Spot
                elif right == 'C':
                    Option_Chain_Filtered['relative_moneyness']  = Spot/Option_Chain_Filtered.index.get_level_values('strike')
                else:
                    raise ValueError(f'Right dne. recieved {right}')
                Option_Chain_Filtered['moneyness_spread'] = (tgt_moneyness-Option_Chain_Filtered['relative_moneyness'])**2
                Option_Chain_Filtered['dte_spread'] = (Option_Chain_Filtered.index.get_level_values('DTE')-tgt_dte)**2
                Option_Chain_Filtered.sort_values(by=['dte_spread','moneyness_spread'], inplace = True)
                Option_Chain_Filtered = Option_Chain_Filtered.loc[Option_Chain_Filtered['dte_spread'] == Option_Chain_Filtered['dte_spread'].min()]
                option_details = Option_Chain_Filtered[(Option_Chain_Filtered['relative_moneyness'] >= tgt_moneyness-moneyness_width) & 
                                                       (Option_Chain_Filtered['relative_moneyness'] <= tgt_moneyness+moneyness_width)]
                option_details['build_date'] = date
                option_details['ticker'] = ticker
                option_details['Spot'] = Spot
                option_details['moneyness'] = tgt_moneyness
                option_details['TGT_DTE'] = tgt_dte
                option_details['q'] = Stock_obj.div_yield()
                option_details['r'] = Stock_obj.rf_rate
                option_details.reset_index(inplace = True)
                option_details.set_index('build_date', inplace = True)
                option_details['right'] = right
                option_details.drop(columns = ['C','P'], inplace = True)
                option_details['option_id'] = option_details.apply(lambda x: generate_option_tick(symbol = x['ticker'], 
                                                                    exp = x['expiration'].strftime('%Y-%m-%d'), strike = float(x['strike']), right = x['right']), axis = 1)
                return_dataframe = pd.concat([return_dataframe, option_details])
            clear_context()
            return_dataframe.drop_duplicates(inplace = True)

        except Exception as e:
            raise
            errors[date] = e
            return errors
        return return_dataframe.sort_values('relative_moneyness', ascending=False),Stock_obj
    else:
        return None, errors
    

# details, obj = chain_details('2024-03-12', 'AAPL', 365, 1, moneyness_width = 0.20)

In [82]:
details

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id
build_date,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,205.0,1.183398,0.033635,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000205P
2024-03-12,2025-03-21,373,200.0,1.154534,0.023881,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000200P
2024-03-12,2025-03-21,373,190.0,1.096808,0.009372,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000190P
2024-03-12,2025-03-21,373,185.0,1.067944,0.004616,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000185P
2024-03-12,2025-03-21,373,180.0,1.039081,0.001527,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000180P
2024-03-12,2025-03-21,373,175.0,1.010218,0.000104,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000175P
2024-03-12,2025-03-21,373,170.0,0.981354,0.000348,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000170P
2024-03-12,2025-03-21,373,165.0,0.952491,0.002257,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000165P
2024-03-12,2025-03-21,373,160.0,0.923628,0.005833,64,AAPL,173.229996,1,365,0.005542,0.0524,P,AAPL20250321000160P


In [155]:
order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.1},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.1}
            ],
            'name': 'vertical_spread'
        }


tick = 'AAPL'
date = '2024-03-12'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'P'

In [209]:
def produce_order_candidates(settings, tick, date, right = 'P'):
    order_candidates = {'long': [], 'short': []}
    for spec in settings['specifics']:
        order_candidates[spec['direction']].append(chain_details(date, tick, spec['dte'], spec['rel_strike'], right,  moneyness_width = spec['moneyness_width'])[0])
    return order_candidates


def liquidity_check(id, date, pass_threshold = 750):
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():

        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]

    start = (pd.to_datetime(date) - BDay(10)).strftime('%Y-%m-%d')
    oi_data = retrieve_openInterest(**transfer_dict, end_date=date, start_date=start)
    return oi_data.Open_interest.mean() > pass_threshold


def available_close_check(id, date):
    sample_id = deepcopy(get_option_specifics_from_key(id))
    new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
    transfer_dict = {}
    for k, v in sample_id.items():
        if k in new_dict_keys:
            if k == 'strike':
                transfer_dict[new_dict_keys[k]] = float(sample_id[k])
            else:
                transfer_dict[new_dict_keys[k]] = sample_id[k]
    start = (pd.to_datetime(date) - BDay(100)).strftime('%Y-%m-%d')
    close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
    close_mask_series = close_data_sample.Close != 0
    return close_mask_series.sum()/len(close_mask_series) 


    
    


In [204]:
available_close_check('AAPL20250321000165P', date)

Datetime
2024-01-30     7.47
2024-01-31     7.95
2024-02-01     8.20
2024-02-02     0.00
2024-02-05     0.00
2024-02-06     7.10
2024-02-07     0.00
2024-02-08     7.05
2024-02-09     0.00
2024-02-12     0.00
2024-02-13     7.80
2024-02-14     7.95
2024-02-15     8.14
2024-02-16     8.10
2024-02-20     8.80
2024-02-21     8.26
2024-02-22     7.95
2024-02-23     0.00
2024-02-26     8.00
2024-02-27     7.70
2024-02-28     8.20
2024-02-29     8.11
2024-03-01     8.35
2024-03-04    10.00
2024-03-05    11.10
2024-03-06    11.33
2024-03-07    11.40
2024-03-08    10.33
2024-03-11    10.45
2024-03-12    10.00
Name: Close, dtype: float64


True

In [151]:
order_candidates = produce_order_candidates(order_settings, tick, date, right)

In [206]:
for direction in order_candidates:
    for data in order_candidates[direction]:
        data['liquidity_check'] = data.option_id.apply(lambda x: liquidity_check(x, date))
        data['available_close_check'] = data.option_id.apply(lambda x: available_close_check(x, date))

In [211]:
order_candidates['long'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check,available_close_check
build_date,,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,190.0,1.096808,0.009372,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000190P,False,False
2024-03-12,2025-03-21,373,185.0,1.067944,0.004616,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000185P,False,False
2024-03-12,2025-03-21,373,180.0,1.039081,0.001527,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000180P,True,True
2024-03-12,2025-03-21,373,175.0,1.010218,0.000104,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000175P,False,True
2024-03-12,2025-03-21,373,170.0,0.981354,0.000348,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000170P,True,True
2024-03-12,2025-03-21,373,165.0,0.952491,0.002257,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000165P,True,True
2024-03-12,2025-03-21,373,160.0,0.923628,0.005833,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000160P,False,True


In [212]:
order_candidates['short'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check,available_close_check
build_date,,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,160.0,0.923628,0.005421,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000160P,False,True
2024-03-12,2025-03-21,373,155.0,0.894764,0.002004,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000155P,False,False
2024-03-12,2025-03-21,373,150.0,0.865901,0.000253,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000150P,True,True
2024-03-12,2025-03-21,373,145.0,0.837037,0.000168,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000145P,False,False
2024-03-12,2025-03-21,373,140.0,0.808174,0.001749,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000140P,False,True
2024-03-12,2025-03-21,373,135.0,0.779311,0.004997,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000135P,False,False
2024-03-12,2025-03-21,373,130.0,0.750447,0.009911,64,AAPL,173.229996,0.85,365,0.005542,0.0524,P,AAPL20250321000130P,False,False


In [219]:
structure = {
    'long': [{'strike': 180.0, 'expiration': '2025-03-21', 'underlier': 'AAPL', 'right': 'p' }],
    'short': [{'strike': 150.0, 'expiration': '2025-03-21', 'underlier': 'AAPL', 'right': 'p' }]
}

struct_obj = OptionStructure(structure)

In [139]:
order_candidates['long'][0]

right,expiration,DTE,strike,relative_moneyness,moneyness_spread,dte_spread,ticker,Spot,moneyness,TGT_DTE,q,r,right,option_id,liquidity_check
build_date,,,,,,,,,,,,,,,
2024-03-12,2025-03-21,373,190.0,1.096808,0.009372,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000190P,False
2024-03-12,2025-03-21,373,185.0,1.067944,0.004616,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000185P,False
2024-03-12,2025-03-21,373,180.0,1.039081,0.001527,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000180P,True
2024-03-12,2025-03-21,373,175.0,1.010218,0.000104,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000175P,False
2024-03-12,2025-03-21,373,170.0,0.981354,0.000348,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000170P,True
2024-03-12,2025-03-21,373,165.0,0.952491,0.002257,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000165P,True
2024-03-12,2025-03-21,373,160.0,0.923628,0.005833,64,AAPL,173.229996,1.0,365,0.005542,0.0524,P,AAPL20250321000160P,False


In [137]:
oi = liquidity_check('AAPL20250321000185P', date)
oi

False

In [177]:
sample_id = deepcopy(get_option_specifics_from_key('AAPL20250321000165P'))
new_dict_keys = {'ticker': 'symbol', 'exp_date': 'exp', 'strike': 'strike', 'put_call': 'right'}
transfer_dict = {}
for k, v in sample_id.items():
    print(k)
    if k in new_dict_keys:
        if k == 'strike':
            transfer_dict[new_dict_keys[k]] = float(sample_id[k])
        else:
            transfer_dict[new_dict_keys[k]] = (sample_id[k])

sample_id

ticker
put_call
exp_date
strike


{'ticker': 'AAPL', 'put_call': 'P', 'exp_date': '2025-03-21', 'strike': 165.0}

In [185]:
close_data_sample = retrieve_eod_ohlc(**transfer_dict, start_date=start, end_date=date)
close_mask_series = close_data_sample.Close != 0
close_mask_series.sum()/len(close_mask_series)
# close_data_sample.Close

0.8

In [210]:
available_close_check('AAPL20250321000160P', date)

0.7333333333333333

In [167]:
close_data_sample

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2024-01-30,7.47,7.47,7.47,7.47,2,6,7.15,134,9.45,8.300,9.351429
2024-01-31,7.95,7.95,7.95,7.95,1,372,6.55,303,9.50,8.025,7.874222
2024-02-01,8.10,8.20,8.10,8.20,2,74,5.55,75,9.35,7.450,7.462752
2024-02-02,0.00,0.00,0.00,0.00,0,108,7.60,258,8.25,7.925,8.058197
2024-02-05,0.00,0.00,0.00,0.00,0,527,7.10,104,7.40,7.250,7.149445
2024-02-06,7.40,7.40,7.05,7.10,38,300,6.85,1,7.00,6.925,6.850498
2024-02-07,0.00,0.00,0.00,0.00,0,726,6.55,167,6.95,6.750,6.624804
2024-02-08,7.05,7.05,7.05,7.05,2,275,7.00,179,7.20,7.100,7.078855
2024-02-09,0.00,0.00,0.00,0.00,0,36,6.85,323,7.10,6.975,7.074930


In [98]:
sample_id = deepcopy(get_option_specifics_from_key('AAPL20250321000160P'))
sample_id

{'ticker': 'AAPL', 'put_call': 'P', 'exp_date': '2025-03-21', 'strike': 160}

In [ ]:
retrieve_openInterest

Signature:
retrieve_openInterest(
    symbol,
    end_date: str,
    exp: str,
    right: str,
    start_date: str,
    strike: float,
    print_url=False,
    proxy='http://18.232.166.224:5500/thetadata',
)
Docstring: Interval size in miliseconds. 1 minute is 6000
File:      ~/cloned_repos/FinanceDatabase/dbase/DataAPI/ThetaData.py
Type:      function

In [37]:

chain_details?

Signature:
chain_details(
    date,
    ticker,
    tgt_dte,
    tgt_moneyness,
    right='P',
    moneyness_width=0.15,
)
Docstring: <no docstring>
File:      /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_16186/3700718713.py
Type:      function

Steps to producing an order:

- S1: RM recieves order settings from PM
- S2: RM produces a dataframe of potential options based on settings (if two legs produce two dataframes)
- S3: RM assesses if option passes all checks
    - C1: Minimum Available close
    - C2: Liquidity (Open Interest)
    - C2.5: (for Spreads only) Ensure both legs are not the same
    - Optional, to extend:
    - C3: Bid-Ask Spread
    
- S4: Return picked order to portfolio manager, which places the order. 
- Example:
    {'long': [optionid or {'strike', 'exp'}], 'short' : []}

In [65]:
generate_option_tick('AAPL','P' ,'2025-03-21', 150.0)
# save_option_keys(generate_option_tick('AAPL','P' ,'2025-03-21', 150.0), {'ticker':'AAPL', 'put_call':'P', 'exp_date':'2025-03-21', 'strike':150.0})

'AAPL20250321000150P'

In [57]:
opt = Option(**option_keys['AAPL20250321000150P'])

Updating db rates data


In [39]:
import_option_keys()
option_keys

{}